# Install

In [1]:
! pip install memory_profiler
%load_ext memory_profiler 

Defaulting to user installation because normal site-packages is not writeable


# Import

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearnex import patch_sklearn  # Speeds up sklearn with intel patch
patch_sklearn()  # Activate patch - changes sklearn imports below

from timeit import default_timer as timer # Time how long commands take
from sklearn.model_selection import train_test_split, StratifiedKFold  # test_train split, cross-validation

from sklearn.experimental import enable_iterative_imputer  # Iterative imputer experimental so need to enable it
from sklearn.impute import IterativeImputer  # Once enabled iterative imputer can be imported

from sklearn.linear_model import RidgeClassifier, BayesianRidge  # Imputation
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder  # Normalisation & Encoding
from imblearn.under_sampling import TomekLinks, RandomUnderSampler   # Undersampling
from imblearn.over_sampling import SMOTENC  # Oversampling
from sklearn.feature_selection import RFE, RFECV  # Recursive feature elimination - feature selection
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier  # RFE
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.svm import SVC
from imblearn.pipeline import Pipeline as imbpipeline

from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import validation_curve

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# Variables

In [3]:
# Number of cores being used 
n_jobs = 10

In [4]:
# Random State
random_state = 14

# General Functions

In [5]:
# Stopwatch to profile function runtimes
class Stopwatch:

    # Initiate constructor
    def __init__(self):
        self.start = timer()
        self.end = None
        self.runtime = None

    # Stop stopwatch
    def stop(self):
        self.end = timer()
        self.runtime = self.end - self.start
        return self.runtime

In [6]:
# Find which columns are categorical and which continuous
def cat_con_cols(df):
    
    columns = [list(df[i]) for i in df]  # Nested list of column values
    num_unique_vals = [len(set([i for i in a if pd.notna(i)])) for a in columns]  # Num of unique values in a column

    categorical_indexes = [i for i, v in enumerate(num_unique_vals) if v <= 100
                           and set([a % 1 for a in df[df.columns[i]].dropna()]) == {0}]

    continuous_indexes = [i for i, v in enumerate(num_unique_vals) if v > 100 or
                          set([a % 1 for a in df[df.columns[i]].dropna()]) != {0}]

    cat = list(df.columns[categorical_indexes])
    con = list(df.columns[continuous_indexes])
    return cat, con

# Data Cleaning 

In [7]:
# Read in data
df = pd.read_csv('/data/home/bt211037/dissertation/supervised_ML_data.tsv',
                   sep='\t', header=0, index_col=0)

### Convert categorical columns to integers

In [8]:
# Get the column names of the continuous and categorical data
cat, con = cat_con_cols(df)  

# Convert categorical cols values from floats to integers 
df[cat] = df[cat].astype('Int64')  

### Separate categorical and Continuous features in dataframe¶

This makes indexing certain features in later processess easier. Continuous features are the first columns followed by categorical.


In [9]:
df = pd.concat([df[con], df[cat]], axis=1, join="inner")

In [10]:
# Column indexes for categorical and continuous features
# Important to exclude target feature - thryoid_cancer
categorical_indexes = [i for i, x in enumerate(df.columns[0:382]) if x in cat]
continuous_indexes = [i for i, x in enumerate(df.columns) if x in con]


# Test Train Split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'thyroid_cancer'],
                                                        df['thyroid_cancer'],
                                                        test_size=0.2,
                                                        shuffle=True,
                                                        random_state=random_state,
                                                        stratify=df['thyroid_cancer'])


# Feature Selection 

Method of choice is Recursive Feature Elimination (RFE) using an ExtraTreesClassifier from sklearn as the estimator. Will be using a subset of the dataset through random undersampling of the majority class for resource efficiency.

### Pipeline

In [12]:
# KNN imputer
knn_imputer = KNNImputer(n_neighbors=1)

In [13]:
# Tomek Links undersampling
tl = TomekLinks(sampling_strategy='majority')

In [14]:
# SMOTE oversampling
smote = SMOTENC(random_state=random_state,
                categorical_features=categorical_indexes,
                sampling_strategy=1)


In [15]:
# Predictive Model 
rfe_model = ExtraTreesClassifier(n_estimators=200,
                                 max_features=20,
                                 max_depth=5,
                                 bootstrap=True,
                                 n_jobs=n_jobs, 
                                 random_state=random_state)

In [16]:
# RFE
rfe = RFE(rfe_model, step=25)

In [17]:
# Final Pipeline
rfe_pipeline = imbpipeline(steps = [('imputer', knn_imputer),
                                ('tomek', tl),
                                ('smotenc', smote),
                                ('rfe', rfe),
                                ('model', rfe_model)])

### Random Undersampling

Randomly undersample majority class to reduce the class imbalance and decrease resource use while retaining the varaiation of the dataset.

Some results are showing that training on this smaller dataset may give better results.

In [18]:
# Configure random undersampler
rus_rfe = RandomUnderSampler(sampling_strategy=0.1,
                         random_state=random_state)

In [19]:
# Create the dataset used for hyperparamter tuning and training
X_res_rfe, y_res_rfe = rus_rfe.fit_resample(X_train, y_train.astype('float64'))


In [20]:
X_res_rfe

,Waist circumference|x48_0_0,Hip circumference|x49_0_0,Standing height|x50_0_0,Time since interview start at which blood pressure screen(s) shown|x96_0_0,Pulse rate automated reading|x102_0_0,Place of birth in UK - north co-ordinate|x129_0_0,Place of birth in UK - east co-ordinate|x130_0_0,Townsend deprivation index at recruitment|x189_0_0,Length of time at current address|x699_0_0,Duration of walks|x874_0_0,...,Genotype measurement batch|x22000_0_0,Genetic sex|x22001_0_0,Genetic kinship to other participants|x22021_0_0,IPAQ activity group|x22032_0_0,Summed days activity|x22033_0_0,Above moderate/vigorous recommendation|x22035_0_0,Above moderate/vigorous/walking recommendation|x22036_0_0,Close to major road|x24014_0_0,Total volume of urine samples held by UKB|x30394_0_0,medication_cbi
0,79.0,99.0,163.1,144.0,89.0,564500.0,425500.0,1.441460,12.0,15.0,...,1,0,1,0,12,0,1,0,5100,-1
1,87.0,90.0,174.0,94.0,69.0,188500.0,514500.0,-0.688732,25.0,100.0,...,0,1,0,0,4,0,1,0,5100,-1
2,68.0,101.0,155.0,138.0,54.0,336500.0,452500.0,-3.168330,28.0,60.0,...,1,0,0,2,19,1,1,0,5100,-1
3,68.0,99.0,159.0,51.0,58.0,388500.0,327500.0,-4.493780,31.0,45.0,...,1,0,0,1,16,1,1,0,5100,-1
4,92.0,102.0,166.0,54.0,65.0,393500.0,381500.0,-2.409450,24.0,30.0,...,1,0,0,2,18,1,1,0,5100,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6034,69.0,89.0,160.0,68.0,65.0,172500.0,534500.0,0.577263,59.0,60.0,...,1,0,1,2,16,1,1,0,5100,-1
6035,68.0,90.0,158.0,101.0,65.0,523500.0,446500.0,-4.338350,19.0,30.0,...,1,0,1,2,12,1,1,0,5100,-1
6036,75.0,88.0,158.0,77.0,74.0,401500.0,386500.0,-3.787730,6.0,240.0,...,1,0,0,<NA>,<NA>,<NA>,<NA>,1,5100,1
6037,60.0,88.0,153.0,53.0,67.0,181500.0,534500.0,-2.833990,1.0,NaN,...,1,0,0,<NA>,<NA>,<NA>,<NA>,0,5100,-1


### Searching for the optimum features

In [21]:
# Parameter to search
rfe_search_grid = {'rfe__n_features_to_select':range(20, 380, 5),
                   'model__max_depth': [5, 10],
                   'model__bootstrap': [True, False]}

In [22]:
# Random search configurations
rf_rfe_hyper_search = RandomizedSearchCV(estimator=rfe_pipeline,
                                    param_distributions=rfe_search_grid,
                                    n_iter=100,
                                    cv=3,
                                    n_jobs=n_jobs,
                                    verbose=1,
                                    random_state=random_state,
                                    scoring='f1')

In [23]:
%%memit

t=Stopwatch()

rf_rfe_hyper_search.fit(X_res_rfe, y_res_rfe)

print(t.stop())

Fitting 3 folds for each of 100 candidates, totalling 300 fits
1806.7079906139988
peak memory: 3460.24 MiB, increment: 755.77 MiB


In [24]:
# Look at results of search
pd.DataFrame(rf_rfe_hyper_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rfe__n_features_to_select,param_model__max_depth,param_model__bootstrap,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,39.004636,0.269763,12.375087,0.250365,340,5,True,"{'rfe__n_features_to_select': 340, 'model__max...",0.198582,0.177215,0.214452,0.196750,0.015257,54
1,37.974652,0.336729,12.810509,0.444442,360,10,True,"{'rfe__n_features_to_select': 360, 'model__max...",0.121429,0.122744,0.155932,0.133368,0.015964,92
2,47.122759,0.636330,11.382344,0.246129,115,10,True,"{'rfe__n_features_to_select': 115, 'model__max...",0.171598,0.200627,0.177143,0.183122,0.012583,62
3,49.776926,0.176523,12.237448,0.779275,45,5,True,"{'rfe__n_features_to_select': 45, 'model__max_...",0.233628,0.207513,0.250804,0.230649,0.017798,1
4,43.483357,0.469439,13.873721,0.498679,250,10,False,"{'rfe__n_features_to_select': 250, 'model__max...",0.134276,0.119298,0.157377,0.136984,0.015663,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,43.663541,0.633547,12.198002,0.369312,285,10,True,"{'rfe__n_features_to_select': 285, 'model__max...",0.149153,0.120301,0.189542,0.152999,0.028398,73
96,46.544633,1.192980,14.141852,0.678622,140,5,False,"{'rfe__n_features_to_select': 140, 'model__max...",0.217742,0.202586,0.252964,0.224431,0.021104,13
97,43.550610,0.893895,11.079278,0.608382,265,5,True,"{'rfe__n_features_to_select': 265, 'model__max...",0.208333,0.186158,0.222707,0.205733,0.015034,44
98,41.141414,0.263103,11.059190,0.496971,310,5,False,"{'rfe__n_features_to_select': 310, 'model__max...",0.235556,0.176471,0.223744,0.211923,0.025529,39


In [25]:
# Find the optimum number of features
rf_rfe_hyper_search.best_params_

{'rfe__n_features_to_select': 45,
 'model__max_depth': 5,
 'model__bootstrap': True}

# Create New Dataset with Selected Features

In [36]:
# Predictive Model 
rfe_model = ExtraTreesClassifier(n_estimators=200,
                                 max_features=20,
                                 max_depth=5,
                                 bootstrap=True,
                                 n_jobs=n_jobs)
                         

In [37]:
# RFE
rfe = RFE(rfe_model, step=25, n_features_to_select=45)

In [38]:
# Extract the optimum feature indexes
# Final Pipeline
rfe_pipeline_feats = imbpipeline(steps = [('imputer', knn_imputer),
                                ('tomek', tl),
                                ('smotenc', smote),
                                ('rfe', rfe)])

In [39]:
# Fit the feature selection pipeline
feats = rfe_pipeline_feats.fit(X_res_rfe, y_res_rfe)

In [40]:
# Get the column names of the selected features
selected_feats = list(X_res_rfe.columns[feats.named_steps['rfe'].support_])

In [41]:
selected_feats

['Whole body fat-free mass|x23101_0_0',
 'Whole body water mass|x23102_0_0',
 'Leg fat percentage (right)|x23111_0_0',
 'Leg predicted mass (right)|x23114_0_0',
 'Leg fat percentage (left)|x23115_0_0',
 'Leg fat-free mass (left)|x23117_0_0',
 'Arm fat percentage (right)|x23119_0_0',
 'Arm fat-free mass (right)|x23121_0_0',
 'Arm predicted mass (right)|x23122_0_0',
 'Arm fat percentage (left)|x23123_0_0',
 'Arm fat-free mass (left)|x23125_0_0',
 'Arm predicted mass (left)|x23126_0_0',
 'Trunk fat-free mass|x23129_0_0',
 'Trunk predicted mass|x23130_0_0',
 'Testosterone|x30850_0_0',
 'Number of days/week walked 10+ minutes|x864_0_0',
 'Number of days/week of vigorous physical activity 10+ minutes|x904_0_0',
 'Salt added to food|x1478_0_0',
 'Alcohol intake versus 10 years previously|x1628_0_0',
 'Father still alive|x1797_0_0',
 'Nervous feelings|x1970_0_0',
 "Tense / 'highly strung'|x1990_0_0",
 "Suffer from 'nerves'|x2010_0_0",
 'Loneliness isolation|x2020_0_0',
 'Seen a psychiatrist fo

In [42]:
selected_feats.append('thyroid_cancer')

In [43]:
# Create new dataframe with only selected features
df_fs = df[selected_feats]

In [44]:
df_fs

,Whole body fat-free mass|x23101_0_0,Whole body water mass|x23102_0_0,Leg fat percentage (right)|x23111_0_0,Leg predicted mass (right)|x23114_0_0,Leg fat percentage (left)|x23115_0_0,Leg fat-free mass (left)|x23117_0_0,Arm fat percentage (right)|x23119_0_0,Arm fat-free mass (right)|x23121_0_0,Arm predicted mass (right)|x23122_0_0,Arm fat percentage (left)|x23123_0_0,...,Attendance/disability/mobility allowance|x6146_0_0,Medication for pain relief constipation heartburn|x6154_0_0,Types of transport used (excluding work)|x6162_0_0,Illnesses of siblings|x20111_0_0,Spirometry QC measure|x20255_0_0,Genetic sex|x22001_0_0,Genetic kinship to other participants|x22021_0_0,Above moderate/vigorous/walking recommendation|x22036_0_0,medication_cbi,thyroid_cancer
userId,,,,,,,,,,,,,,,,,,,,,
1000020,42.0,30.8,32.5,6.9,33.4,7.1,30.1,2.0,1.9,29.7,...,0,-1,1,0,1,0,0,1,-1,0
1000037,40.4,29.6,43.9,6.3,43.1,6.8,39.6,2.0,1.9,40.7,...,0,5,2,8,3,0,0,<NA>,-1,0
1000043,47.1,34.5,40.8,7.5,41.5,7.8,37.6,2.4,2.2,37.9,...,0,-1,1,8,1,0,0,<NA>,-1,0
1000066,65.9,48.2,19.6,9.7,20.4,10.0,17.1,3.8,3.6,14.9,...,0,-1,1,<NA>,3,1,0,<NA>,-1,0
1000092,46.2,33.8,45.6,7.7,45.5,8.1,45.7,2.4,2.2,47.1,...,2,1,1,0,<NA>,0,0,1,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6025060,75.7,55.4,27.6,12.1,23.5,13.3,27.7,4.5,4.2,28.0,...,0,-1,1,0,1,1,0,1,2,0
6025078,57.4,42.0,17.4,9.2,17.6,9.5,18.1,3.2,3.0,20.0,...,0,-1,3,<NA>,3,1,0,1,-1,0
6025087,58.8,43.1,35.7,9.5,33.4,10.2,34.8,3.3,3.1,35.6,...,0,2,1,<NA>,<NA>,1,0,1,2,0


In [45]:
# Save new dataframe (tsv) with only the selected features 
# To be used as the dataset from now on
df_fs.to_csv('/data/home/bt211037/dissertation/feats_selected_dataset.tsv',
             sep='\t')  
